<a href="https://colab.research.google.com/github/arezaz/nlp-workshop/blob/master/IndoDataWeek_NLP_Alireza_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**The Natural Language Processing Workshop**
---
#### Alireza Rezazadeh (rezaz003@umn.edu)
Indo Data Week - November, 2020


## **1. Word Representation**

#### **Overview**


1. Word Representation:
  * 1.1. Basic Approaches
      * 1.1.1. One-Hot Encoding
      * 1.1.2. Bag of Words
      * 1.1.3. TF-IDF
  * 1.2. Word Embedding
      * 1.2.1. Word2Vec
      * 1.2.2. GloVe
      

#### **1.1. Basic Aproaches**


*Vector space model* is an algebraic model of representing text as vecotrs of identifiers. All following approaches discussed here are one way or another based on the idea of vector space models. Basic idea:

<img src="https://raw.githubusercontent.com/arezaz/nlp-workshop/master/contents/1.png" alt="Drawing" width="350" />

The most basic idea: assigning unique ID to each word in the vocabulary. This way, we can represent text as a multi-dimensional vector. Let's go over a few ways to implement this idea!

##### **1.1.1. One-Hot Encoding**
<img src="https://raw.githubusercontent.com/arezaz/nlp-workshop/master/contents/2.png" alt="Drawing" width="700" />

Let's try this!

In [ ]:
VOCAB = {'yellow':0, 'apple':1, 'dog':2, 'cat':3, 'blue':4, 'black':5, 'sky':6}

In [ ]:
def onehot(text):
  onehot_rep = []
  for word in text.split():
    # initializing one-hot vector
    vec = [0]*len(VOCAB)
    if word in VOCAB.keys():
      # building one-hot vector
      vec[VOCAB[word]] = 1
    onehot_rep.append(vec)
  return onehot_rep

In [ ]:
onehot('the sky is blue and the cat is black')

[[0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0]]

Why is this approach not so practical?
  * High dimensionality of the representation.
  * Sparsity leads to computational difficaulties.
  * Does not capture semantic relation between words.




##### **1.1.2. Bag of Words**

<img src="https://raw.githubusercontent.com/arezaz/nlp-workshop/master/contents/3.png" alt="Drawing" width="700" />

Let's try this!

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [ 'This is the first document and it is interesting.',
           'This document is the second document and it is in English.',
           'And this is the third one.',
           'Is this the first document?']


count_vect = CountVectorizer() # CountVectorizer(binary=True)

# Building bag-of-word representation of the corpus
bow_rep = count_vect.fit_transform(corpus)

# Extracting the vocabulary
print("Our vocabulary: ", count_vect.vocabulary_)

# Bag-of-words representation of documents
print("BoW of '" + corpus[0] + "': ", bow_rep[0].toarray())
print("BoW of '" + corpus[1] + "': ",bow_rep[1].toarray())

Our vocabulary:  {'this': 12, 'is': 6, 'the': 10, 'first': 3, 'document': 1, 'and': 0, 'it': 7, 'interesting': 5, 'second': 9, 'in': 4, 'english': 2, 'third': 11, 'one': 8}
BoW of 'This is the first document and it is interesting.':  [[1 1 0 1 0 1 2 1 0 0 1 0 1]]
BoW of 'This document is the second document and it is in English.':  [[1 2 1 0 1 0 2 1 0 1 1 0 1]]


In [ ]:
# Bag-of-words representation of a new documents
NewDocument = "dog and dog are friends"
temp = count_vect.transform([NewDocument])
print("BoW of '" + NewDocument + "': ",temp.toarray())

BoW of 'dog and dog are friends':  [[1 0 0 0 0 0 0 0 0 0 0 0 0]]


A few advantages:
  *   Straight-forward and easy to implement.
  *   Captures semantic similarity for documents.
  *   Encoding is fixed length for any document.

Some drawbacks:
  *   Vectors could become sparse for larger vocabulary.
  *   Blind! cannot capture semantic similarity of words.
  *   What if a new word is not in the vocabulary?
  *   It's a bag! disregards words order information.

##### **1.1.3. TF-IDF**
 

<img src="https://raw.githubusercontent.com/arezaz/nlp-workshop/master/contents/4.png" alt="Drawing" width="1400" />

Let's try this!

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

tfidf = TfidfVectorizer()
bow_rep_tfidf = tfidf.fit_transform(corpus)

df_temp = pd.DataFrame({'Vocab': tfidf.get_feature_names(), 'IDF':tfidf.idf_}) # features and their IDF
df_temp

,Vocab,IDF
0,and,1.223144
1,document,1.223144
2,english,1.916291
3,first,1.510826
4,in,1.916291
5,interesting,1.916291
6,is,1.000000
7,it,1.510826
8,one,1.916291
9,second,1.916291


In [ ]:
NewDocument = "This English document is interesting"
temp = tfidf.transform([NewDocument])
print("TF-IDF for '"+NewDocument+"' :\n", temp.toarray())

TF-IDF for 'This English document is interesting' :
 [[0.         0.37149619 0.58202057 0.         0.         0.58202057
  0.30372248 0.         0.         0.         0.         0.
  0.30372248]]


There are a few variations of TF-IDF formulations. Advantages of this approach are:
*   Like bag-of-words, it captures semantic similarity for documents.
*   Very commonly used (even today!).

But, like any other approach, there are some drawbacks:
*   Curse of dimensionality! feature vectors can become very high dimensional.
*   Still doesn't capture semantic relation between words.
*   Sparsity in feature vecotors.
*   Does not handle out-of-vocabulary words.



#### **1.2. Word Embeddings**

Words that can be used interchangably often have a strong semantic relation. 

> The [*cat*, *dog*] is a [*domestic*, *wild*] species of [*small*, *large*] carnivorous mammal.  - *Wikipedia*

Word embeddings represent words as real-valued vectors in a predifined vector space. In particular, word embeddings are from the class of dense distributed representation. Unlike sparse representations, such as one-hot encoding, distributed representations are learned based on the usage of words. 

##### **1.2.1. Word2Vec**
 

<img src="https://raw.githubusercontent.com/arezaz/nlp-workshop/master/contents/5.png" alt="Drawing" width="1700" />

Word embedding models are usually trained on huge datasets with large vocabulary size. Luckily we don't need to train neural networks from scratch Let's try a pre-trained model!

In [ ]:
from gensim.models import Word2Vec 
from gensim.test.utils import common_texts

# Build a Word2Vec model on a costume dataset
model = Word2Vec(common_texts, size=10, window=5, min_count=1, workers=4)

In [ ]:
# Finding similar words
w = 'human'
print(w + ' is similar to: ' + str(model.wv.most_similar(w, topn=3)))

# the representation for one of the words
print(w + ' vector is: ' + str(model[w]))

human is similar to: [('computer', 0.7257747650146484), ('system', 0.3190498352050781), ('interface', 0.20291012525558472)]
human vector is: [ 0.0156147   0.04177426  0.02856586 -0.03033977  0.01721545 -0.01652424
  0.0282451   0.02292306  0.01094562  0.04534782]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


##### **1.2.2. GLoVe**
 

<img src="https://raw.githubusercontent.com/arezaz/nlp-workshop/master/contents/6.png" alt="Drawing" width="1700" />

Let's try a pre-trained GLoVe model based on twitter data!

In [ ]:
import gensim.downloader

# loading GloVe model trained on tweets
glove_model = gensim.downloader.load('glove-twitter-25')

# Finding similar words
w = 'computer'
print(w + ' is similar to: ' + str(glove_model.most_similar(w, topn=5)))

[==================================================] 100.0% 104.8/104.8MB downloaded
computer is similar to: [('camera', 0.907833456993103), ('cell', 0.891890287399292), ('server', 0.8744666576385498), ('device', 0.869352400302887), ('wifi', 0.8631256818771362)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


To measure similarity between two vectors we can use *cosine similarity* metric. It is defined to equal the cosine of the angle between them, which is also the same as the inner product of the same vectors normalized to both have length 1.

<img src="https://www.oreilly.com/library/view/mastering-machine-learning/9781785283451/assets/d258ae34-f4f8-4143-b3c2-0cb10f2b82de.png" alt="Drawing" width="800" />


In [ ]:
glove_model[w]

array([ 0.64005 , -0.019514,  0.70148 , -0.66123 ,  1.1723  , -0.58859 ,
        0.25917 , -0.81541 ,  1.1708  ,  1.1413  , -0.15405 , -0.11369 ,
       -3.8414  , -0.87233 ,  0.47489 ,  1.1541  ,  0.97678 ,  1.1107  ,
       -0.14572 , -0.52013 , -0.52234 , -0.92349 ,  0.34651 ,  0.061939,
       -0.57375 ], dtype=float32)

In [ ]:
from scipy import spatial

# similarity function based on cosine similarity
def similarity(w1,w2):
  return 1 - spatial.distance.cosine(glove_model[w1],glove_model[w2])
  

In [ ]:
w1s, w2s = 'man', 'woman'
w1d, w2d = 'man', 'umbrella'

print(f"similarity({w1s}, {w2s}) = ", similarity(w1s, w2s))
print(f"similarity({w1d}, {w2d}) = ", similarity(w1d, w2d))

similarity(man, woman) =  0.7654176354408264
similarity(man, umbrella) =  0.4887590706348419


In [ ]:
w1s, w2s = 'small', 'smaller'
w1d, w2d = 'large', 'larger'

print(f"similarity({w1s}, {w2s}) = ", similarity(w1s, w2s))
print(f"similarity({w1d}, {w2d}) = ", similarity(w1d, w2d))

similarity(small, smaller) =  0.8645442128181458
similarity(large, larger) =  0.8587130904197693


This is interesting. But wait! does data equally favor everyone? What if we want to somehow use a model trained on a large dataset of tweets to make a decision for the society? 

Let's take a look at how language can be unfair and biased! 

In [ ]:
career = 'doctor'

male_names = ['john', 'carl', 'steven', 'ashton', 'alireza', 'mohammed', 'ishaan']
female_names = ['rose', 'emma', 'lily', 'kylie', 'yasmin', 'zahra', 'shanaya']

print('~~~~~~~~~ Male names ~~~~~~~~~')
for name in male_names:
  print(f"s({name}, {career}) ---> {round(similarity(name, career),3)}")

print('\n~~~~~~~~~ Female names ~~~~~~~~~')
for name in female_names:
  print(f"s({name}, {career}) ---> {round(similarity(name, career),3)}")

~~~~~~~~~ Male names ~~~~~~~~~
s(john, artist) ---> 0.686
s(carl, artist) ---> 0.549
s(steven, artist) ---> 0.595
s(ashton, artist) ---> 0.573
s(alireza, artist) ---> 0.039
s(mohammed, artist) ---> 0.276
s(ishaan, artist) ---> -0.128

~~~~~~~~~ Female names ~~~~~~~~~
s(rose, artist) ---> 0.575
s(emma, artist) ---> 0.649
s(lily, artist) ---> 0.588
s(kylie, artist) ---> 0.667
s(yasmin, artist) ---> 0.331
s(zahra, artist) ---> 0.283
s(shanaya, artist) ---> -0.113


We experimented with one of the common word embedding models and observed how based on biased data a machine learning model can be biased. Recently, the subject of AI fairness has been attracting more attention.

---


## **2. Language Models**

#### **Overview**

2. Language Models
  * 2.1 Basic Models
      * 2.1.1. Unigram/Bigram Models
      * 2.1.2. N-grams Models
  * 2.2 Recurrent Nerual Networks (RNN)
      * 2.2.1. Basic RNN
      * 2.2.2. Vanilla RNN
      * 2.2.3. LSTM
      * 2.2.4. Deep RNN

* Hands-on: Building a Language Model

#### **2.1. Basic Models**


Language models, originally developed for speech recognition, are widely used in many NLP applications. Probabilistic language models compute the probability of a sentence or sequence of words. Specifically, language models can be developed to predict the likelihood of a given word, or a sequence of words, to follow a sequence of words.

##### **2.1.1. Unigram/Bigram Models**

<img src="https://raw.githubusercontent.com/arezaz/nlp-workshop/master/contents/7.png" alt="Drawing" width="1700" />

##### **2.1.2. N-gram Models**

<img src="https://raw.githubusercontent.com/arezaz/nlp-workshop/master/contents/8.png" alt="Drawing" width="1800" />


In [ ]:
import nltk
from nltk.util import ngrams
nltk.download('punkt')


# function to extract n-grams of sentences
def ngrams_gen(seq, n):
    n_grams = ngrams(nltk.word_tokenize(seq), n)
    return [ ' '.join(grams) for grams in n_grams]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
sentence = 'The whole is more than the sum of its parts' # -Aristotle

n = [1, 2, 3, 4]
for _ in n:
  print(str(_)+"-gram: ", ngrams_gen(sentence, _))

1-gram:  ['The', 'whole', 'is', 'more', 'than', 'the', 'sum', 'of', 'its', 'parts']
2-gram:  ['The whole', 'whole is', 'is more', 'more than', 'than the', 'the sum', 'sum of', 'of its', 'its parts']
3-gram:  ['The whole is', 'whole is more', 'is more than', 'more than the', 'than the sum', 'the sum of', 'sum of its', 'of its parts']
4-gram:  ['The whole is more', 'whole is more than', 'is more than the', 'more than the sum', 'than the sum of', 'the sum of its', 'sum of its parts']


In general, N-gram models are not sufficient models for language altough they are very useful. Language is essentially dependent on complicated long-distance histories.

#### **2.2. Recurrent Neural Networks**

As mentioned earlier, language is sequential at its essense. *Recurrent neural networks* (RNN) are specifically developed to capture a sequential process. In general, RNN is composed of units that keep a memory of preceeding history. This memory is temporal as the units keep updating at every timestep. RNN are very promising in many NLP applications.


*images courtesy of cs230 - stanford, and colah.github.io*

##### **2.2.1. Basic RNN**

At each timestep $t$, the unit takes input $x^{<t>}$, outputs $y^{<t>}$, and pass a signal $a^{<t>}$ based on its computations to the next timestep $t+1$. This signal acts as a memory in the sequence.


<img src="https://stanford.edu/~shervine/teaching/cs-230/illustrations/architecture-rnn-ltr.png?9ea4417fc145b9346a3e288801dbdfdc" alt="Drawing" width="600" />

Specifically, 

$a^{<t>}=g_a(W_a [a^{<t-1>}, x^{<t>}]+b_a)$

$\hat{y}^{<t>}=g_y(W_y a^{<t>}+b_y)$

The loss function is defined as: $L(\hat{y}-y) = \sum_{t=1}^{T}L(\hat{y}^{<t>}-y^{<t>}) $. The RNN network is trained by *backpropagation through time*.

Type | Schematic | Application
 --- |    ---    |     ---
one-to-one | <img src="https://stanford.edu/~shervine/teaching/cs-230/illustrations/rnn-one-to-one-ltr.png?9c8e3b04d222d178d6bee4506cc3f779" alt="Drawing" width="400" /> | a basic nerual network 
one-to-many | <img src="https://stanford.edu/~shervine/teaching/cs-230/illustrations/rnn-one-to-many-ltr.png?d246c2f0d1e0f43a21a8bd95f579cb3b" alt="Drawing" width="400" /> | music generation
many-to-one | <img src="https://stanford.edu/~shervine/teaching/cs-230/illustrations/rnn-many-to-one-ltr.png?c8a442b3ea9f4cb81f929c089b910c9d" alt="Drawing" width="400" /> | sentiment classification
many-to-many | <img src="https://stanford.edu/~shervine/teaching/cs-230/illustrations/rnn-many-to-many-same-ltr.png?2790431b32050b34b80011afead1f232" alt="Drawing" width="400" /> | named-entity recognition (NER)
many-to-many | <img src="https://stanford.edu/~shervine/teaching/cs-230/illustrations/rnn-many-to-many-different-ltr.png?8ca8bafd1eeac4e8c961d9293858407b" alt="Drawing" width="400" /> | machine translation

##### **2.2.2. Vanilla RNN**

Vanilla RNN is a basic RNN unit. The non-linearity function is set to $tanh$ and can be described as:

$a^{<t>}=tanh(W_a[a^{<t-1>}, x^{<t>}]+b_a)$



<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-SimpleRNN.png" alt="Drawing" width="600" />

##### **2.2.3. LSTM**

LSTM (long short-term memory) network introduced back in 1990s are extremely promising in a wide variety of sequential modelings. They are formulated to allow for learning lags of unknown duration between relevant events in a sequential data. A major feature of LSTMs is their capability in experssing the notion of forgetting.

There are many variations of LSTMs. In general, an LSTM cell has three main components, also called *gates*: input gate, output gate, and forget gate. On top of that, each cell has a *state* associated with it.


<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png" alt="Drawing" width="600" />

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM2-notation.png" alt="Drawing" width="500" />


So, what is the main idea?

The cell state runs through all timesteps and the cell can add or remove information to the state signal. The three gates allows the cell to arbitrarily let information through. Gates are composed of sigmoid function to determine how much of each component let through. 

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-C-line.png" alt="Drawing" width="700" />


Let's briefly walk through an LSTM cell:

*1) Decide what to forget:* look at the previous timestep hidden state $h_{t-1}$ and the input at this timestep $x_t$ and through a sigmoid layer determine how much to forget *(forget gate)*: 

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-f.png" alt="Drawing" width="700" />

*2) Decide what to remember:* look at the previous timestep hidden state $h_{t-1}$ and the input at this timestep $x_t$. First, decide which values to update through a sigmoid layer *(input gate)*. Then, compute the candidate values for the updates: 

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-i.png" alt="Drawing" width="700" />

*3) Update cell states:* now that we know what to remember and what to forget we can impose them to the cell state signal: 

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-C.png" alt="Drawing" width="700" />

*4) Come up with an output:* look at the previous timestep hidden state $h_{t-1}$ and through a sigmoid layer decide what parts of this signal to update and output (*output gate*). Then, combine this signal with the candidate update values and compute the hidden state at this timestep $h_{1}$: 

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-o.png" alt="Drawing" width="700" />

As mentioned before, LSTMs are great and promising in general but we should also be aware of some of their drawbacks:

* LSTMs are developed to solve the problem of vanishing gradients in the process of backpropagation. However, they also suffer from vanishing gradient problem to some extent.
* The cell function is quite complex and this can exacerbate the issue of vanishing gradients.
* Computationally expensive to train up to a decent performance level.
* Cannot remember history of sequence for relatively longer timesteps.

##### **2.2.4. Deep RNN**

How do we make an RNN deep? just stack them up and build layers on top of each other!

<img src="https://stanford.edu/~shervine/teaching/cs-230/illustrations/deep-rnn-ltr.png?f57da6de44ddd4709ad3b696cac6a912" alt="Drawing" width="400" />


---

## 3. Hands-on: Working with Transformers



### So, what are the Transformers really?

Transformer in natural language processing refers to state of the art sequence-to-sequence language models. The backbone of transformers is *attention mechanism* to handle long distance dependencies. 

Attention mechanism basically weights different positions of a single sequence in order to compute a representation of the sequence. Here's how it looks:

<img src="https://miro.medium.com/max/700/1*wa4zt-LcMWRIYLfiHfBKvA.png" alt="Drawing" width="450" />




### 3.1. GPT-2
Generative  Pre-trained Transformers (GPT) are developed by OpenAI. GPT-2 is released in 4 different variations based on their size (small, medium, large, and XL). GPT-2 is a very large model with 1.5 billion parameters. The pre-trained models contain information from 8 million web pages from. GPT-2 is an unsupervised language model announced in 2019. Building blocks of GPT-2 is:

<img src="https://camo.githubusercontent.com/795bd8868fdeb49b7ca48a935e806b56169a172b/68747470733a2f2f692e696d6775722e636f6d2f305853535842642e706e67" alt="Drawing" width="180" />


#### 3.1.1. Next Word Generation Using GPT-2



In [ ]:
!pip install pytorch-transformers

     |████████████████████████████████| 184kB 2.7MB/s 
     |████████████████████████████████| 1.1MB 9.1MB/s 
     |████████████████████████████████| 133kB 17.6MB/s 
     |████████████████████████████████| 890kB 11.4MB/s 
     |████████████████████████████████| 6.8MB 16.9MB/s 
     |████████████████████████████████| 71kB 6.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=b4399bee5825c78a01ba9643a8c3e825de7c8dcd995add8eb926e7f5bf348cc3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.19.16 has requirement urllib3<1.26,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
# importing GPT2 library
import torch
from pytorch_transformers import GPT2Tokenizer, GPT2LMHeadModel

In [ ]:
# creating an instance of GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

100%|██████████| 456318/456318 [00:00<00:00, 6607595.92B/s]


In [ ]:
# loading a pre-trained GPT-2 model with multi-head attention mechanism
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.eval()
model

100%|██████████| 548118077/548118077 [00:13<00:00, 42113603.07B/s]


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [ ]:
# let's predict the next word in this sequence
text = "artificial inteligence aims to predict the"
indexed_tokens = tokenizer.encode(text)
tokens_tensor = torch.tensor([indexed_tokens])
print(f'tokens are: {indexed_tokens}')

tokens are: [11666, 33649, 328, 594, 12031, 284, 4331, 262]


In [ ]:
# Predicting the next word in the sequence
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]

In [ ]:
predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])
predicted_text

' artificial inteligence aims to predict the future'

#### 3.1.2. Next Sentences Generation Using GPT-2



In [ ]:
start = 'Intelligence is most often studied in humans but has also been observed in both'
indexed_tokens = tokenizer.encode(start)

for i in range(150):
  tokens_tensor = torch.tensor([indexed_tokens])
  with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]
    predicted_index = torch.argmax(predictions[0, -1, :]).item()
    indexed_tokens = indexed_tokens + [predicted_index]

In [ ]:
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])
print(predicted_text)

 Intelligence is most often studied in humans but has also been observed in both animals and humans.

The study, published in the journal Nature Communications, found that the brain of a human with autism was more sensitive to the presence of a chemical called methylphenidate, which is found in the brain of a human with autism.

"This is the first time that we have found that the brain of a human with autism is more sensitive to methylphenidate than that of a human with autism," said lead author Dr. David H. Hirsch, a professor of psychiatry at the University of California, San Francisco.

The study was conducted in a laboratory at the University of California, San Francisco.

The researchers found that the brain of a human with autism was more sensitive to methylphenidate than that that


---
## Relevant Materials
* Vajjala, Majumder, Gupta, Surana - [Practical Natural Language Processing: A Comprehensive Guide to Building Real-World NLP Systems](https://www.amazon.com/Practical-Natural-Language-Processing-Pragmatic/dp/1492054054)
* [Sequence Models](https://www.coursera.org/learn/nlp-sequence-models) by deeplearning.ai
* [Stanford's CS230](https://stanford.edu/~shervine/teaching/cs-230/cheatsheet-recurrent-neural-networks) course materials
* Christopher Olah's [blog](https://colah.github.io/)
